### Task 1 - WikiParser

In [22]:
import sys, re
sys.path.append('/users/Sofia/desktop/python-hw/homework_2/pattern-2.6') # specific for my machine

from pattern.web import Wikipedia

class WikiParser:
    def __init__(self):
        pass
    
    def __get_text(self, search_term):
        
        article = Wikipedia().search(search_term)
        if article:
            s = article.string        
            if s:
                while '\n' in s:
                    s = s.replace('\n', ' ')
                while '  ' in s:
                    s = s.replace('  ', ' ')

                #s = re.sub(r'[^\w\s]','',s) # strips punctuation
                # оставить знаки пунктуации, которые разделяют предложения
                s = re.sub(r'[^\w\s^\.^!^?]','',s)
                s = s.lower()
                return s
    
            else:
                return None

        else:
            return None
    
    def get_articles(self, search_term):
        text_list = []
        
        article = Wikipedia().search(search_term)
        links = article.links
        
        s = self.__get_text(search_term)
        text_list.append(s)
        
        for link in links:
            text_list.append(self.__get_text(link))
            print(link)

        return text_list


### Task 2 - TextStatistics

In [51]:
from nltk import ngrams
from collections import Counter
import operator
import math

class TextStatistics:
    def __init__(self, articles):        
        self.articles = articles
        pass

    def get_top_3grams(self, n, use_idf=True):
        
        all_three_grams = []
        
        for article in self.articles:
            #words = article.split()      
            words = list(article)        # split by symbol
             
            #for word in words:          keep digits
            #    if word.isdigit():
            #        words.remove(word)
                    
            while '' in words:
                words.remove('')

            three_grams = ngrams(words, 3)
            for gram in three_grams:
                all_three_grams.append(gram)

# use_idf, при выставлении которого частоты умножаются на коэффициент IDF и соотвественно сортируются.
# IDF = log (<общее кол-во предложений в корпусе> / <кол-во предложений, в которых встретилась данная триграмма>)

        if use_idf:

            # get num of sentences (punctuations)
            whole_text = ' '.join(self.articles)
            sents = re.split(r' *[\.\?!][\'"\)\]]* *', whole_text)
            
            freq_list = {}
            
            for g in all_three_grams:
                count_occurance = 0
                for s in sents:
                    s = str(s)
                    if str(g) in s:
                        count_occurance += 1
                
                if count_occurance > 0:
                    idf = math.log(len(sents) / count_occurance)
                else:
                    idf = 0
                    
                freq_list[g] = idf
                
            freq_list = sorted(freq_list.items(), key=operator.itemgetter(1))
            freq_list = freq_list[0:n-1]
            
        else:
        
            freq_list = Counter(tuple(i) for i in all_three_grams)
            freq_list = freq_list.most_common(n)

        list_of_3grams_in_descending_order_by_freq = []
        list_of_their_corresponding_freq = []
        
        for i in freq_list:
            list_of_3grams_in_descending_order_by_freq.append(i[0])
            list_of_their_corresponding_freq.append(i[1])
            
        return (list_of_3grams_in_descending_order_by_freq, list_of_their_corresponding_freq)
    
    def get_top_words(self, n, use_idf=True):
        
        all_words, bad_list = [], ['a', 'an', 'the', 'aboard', 'about', 'above', 'across', 'between',
                                   'afore', 'after', 'against', 'along', 'amid', 'amidst', 'among',
                                   'amongst', 'around', 'as', 'aside', 'aslant', 'astride', 'at',
                                   'athwart', 'atop', 'before', 'behind', 'below', 'beneath', 'beside',
                                   'besides', 'between', 'betwixt', 'beyond', 'by', 'circa', 'despite',
                                   'down', 'except', 'for', 'from', 'in', 'inside', 'into', 'like', 'near',
                                   'neath', 'next', 'of', 'off', 'on', 'opposite', 'out', 'outside', 'over',
                                   'per', 'through', 'till'', toward', 'towards', 'under', 'underneath',
                                   'unlike', 'until', 'up', 'with', 'without']
        
        for article in self.articles:
                      
            words = article.split()
            for word in words:
                k = 0
                for b in bad_list:
                    if word == b:
                        k += 1
                        
                if k == 0: 
                    if not word.isdigit(): 
                            if word != '':
                                all_words.append(word)
        
        if use_idf: # added idf parameter     
            
            word_list = list(set(all_words))
            word_count = {}
            
            for i in word_list:
                word_occurances = 0
                word = i[0]
                for a in self.articles:
                    if word in a:
                        word_occurances += 1
                
                if word_occurances > 0:
                    idf = math.log(len(self.articles) / word_occurances)
                    
                else:
                    idf = 0
                    
                word_count[word] = idf
                
            # sort by most common, get top n
            word_count = sorted(word_count.items(), key=operator.itemgetter(1))
            word_count = word_count[0:n-1]
            
                
        else:
            word_count = Counter(word for word in all_words)
            word_count = word_count.most_common(n)

        list_of_words_in_descending_order_by_freq = []
        list_of_their_corresponding_freq = []
        
        for i in word_count:
            list_of_words_in_descending_order_by_freq.append(i[0])
            list_of_their_corresponding_freq.append(i[1])
        
        return (list_of_words_in_descending_order_by_freq, list_of_their_corresponding_freq)


### Task 3 - Experiment

In [52]:
from tabulate import tabulate

#from task_1 import Wikiparser
#from task_2 import TextStatistics

class Experiment:
    def __init__(self):
        pass
    
    def __phrase_print(self, term_list, freq_list):
        
        data = []
        for a, b in zip(term_list, freq_list):
            add = [' '.join(a), str(b)]
            data.append(add)
        print tabulate(data)
        
    def __word_print(self, term_list, freq_list):
        
        data = []
        for a, b in zip(term_list, freq_list):
            add = [''.join(a), str(b)]
            data.append(add)
        print tabulate(data)
    
    def show_results(self):
        
        a = WikiParser()
        nlp_corpus = a.get_articles('Natural_language_processing')
        nlp_text = [nlp_corpus[0]]

        b = TextStatistics(nlp_text)

        nlp_three_grams = b.get_top_3grams(5)
        print('top 5 NLP page 3-grams:\n')
        self.__phrase_print(nlp_three_grams[0], nlp_three_grams[1])

        nlp_top_words = b.get_top_words(5)
        print('\ntop 5 NLP page words:\n')
        self.__word_print(nlp_top_words[0], nlp_top_words[1])

        c = TextStatistics(nlp_corpus)    

        three_grams = c.get_top_3grams(20)
        print('\ntop 20 NLP Corpus 3-grams:\n')
        self.__phrase_print(three_grams[0], three_grams[1])

        top_words = c.get_top_words(20)
        print('\ntop 20 NLP Corpus words:\n')
        self.__word_print(top_words[0], top_words[1])





#### Выполнить парсинг статей википедии для "Natural language processing"
#### По полученному корпусу текстов посчитать топ-20 3-грамм и топ-20 слов с use_idf=True.

In [54]:
#e = Experiment()
#texts = e.show_results()